In [1]:
import sys
# sys.path.append('/Users/bernardoloureiro/template-lib')

from utils.notebookhelpers.helpers import Helpers
from utils.dtos.templateOutputCollection import TemplateOutputCollection
from utils.dtos.variable import Metadata
from utils.dtos.templateOutput import TemplateOutput
from utils.dtos.templateOutput import OutputType
from utils.dtos.templateOutput import ChartType
import datetime
import logging
import pandas as pd
import numpy as np
import os
from dateutil import parser
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)
import math
import scipy as scipy
import matplotlib.pyplot as plt
import seaborn as sns
import copy
%matplotlib inline
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import ADASYN
from sklearn.ensemble import RandomForestClassifier

from collections import Counter
from sklearn.linear_model import LogisticRegressionCV
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve, auc
# from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.calibration import CalibratedClassifierCV

In [2]:
inputDatasetParameter=Helpers.get_or_create_input_dataset(
    name="inputBeneDataset2",
    metadata=Metadata(input_name='Bene_df', is_required=True, tooltip='Benefeciary Dataset for EDA'),
    local_context=locals()
)

# Gender = Helpers.get_or_create_input_var(
#     name="Gender",
#     metadata=Metadata(input_name="Longitude", is_required=True, tooltip="Longitude", multiple=False, datatypes=["DOUBLE", "LONG"], options=['FIELDS', 'CONSTANT'], datasets=['GeoDataset']),
#     local_context=locals()
# )

EDA_Beneficiary=Helpers.get_or_create_output_chart(
 name="EDA_Beneficiary_Data",
    metadata=Metadata(input_name='Gender_chart Chart Name', is_required=True, tooltip='Name of the chart to be created'),
    local_context=locals()
)

In [6]:
contextId = "EDA_Beneficiary_Data"
context = Helpers.getOrCreateContext(contextId = contextId, localVars=locals())

In [7]:
entities=Helpers.getAllEntities(context)
BeneDataset = inputDatasetParameter.value

EDA_Beneficiary_name=EDA_Beneficiary.value

KeyError: 'entityPaths'

In [ ]:
df = Helpers.getEntityData(context,BeneDataset)

In [ ]:
train_bene_df = df.copy()

In [2]:
pd.set_option('display.max_columns',30)
label_font_dict = {'family':'sans-serif','size':13.5,'color':'brown','style':'italic'}
title_font_dict = {'family':'sans-serif','size':16.5,'color':'Blue','style':'italic'}

In [ ]:
train_bene_df['Gender'] = train_bene_df['Gender'].apply(lambda val: 0 if val == 2 else 1)

In [ ]:
# Here, I'm displaying the distribution of BENEFICIARIES on the basis of GENDER 
with plt.style.context('seaborn'):
    plt.figure(figsize=(10,8))
    fig = train_bene_df['Gender'].value_counts().plot(kind='bar', color=['yellow','purple'])
    # Using the "patches" function we will get the location of the rectangle bars from the graph.
    ## Then by using those location(width & height) values we will add the annotations
    for p in fig.patches:
        width = p.get_width()
        height = p.get_height()
        x, y = p.get_xy()
        fig.annotate(f'{str(round((height*100)/train_bene_df.shape[0],2))+"%"}', (x + width/2, y + height*1.015), ha='center', fontsize=13.5)
    # Providing the labels and title to the graph
    plt.xlabel("Gender Code", fontdict=label_font_dict)
    plt.ylabel("Number or % share of patients\n", fontdict=label_font_dict)
    plt.grid(which='major', linestyle="--", color='lightgrey')
    plt.minorticks_on()
    plt.title("Distribution of BENEFICIARIES based on GENDER\n", fontdict=title_font_dict)

In [ ]:
outputCollection = Helpers.createOutputCollection(context)
imgChart = Helpers.createTemplateOutputPlotlibChart(context, chartTitle='Distribution of BENEFICIARIES based on GENDER', plt=plt)

outputCollection.addTemplateOutput(imgChart)

In [ ]:
Helpers.save(context)

### **Q3. Lets calculate the AGE of every BENEFICIARY?**

In [4]:
train_bene_df['DOB'] = pd.to_datetime(train_bene_df['DOB'], format="%Y-%m-%d")

NameError: name 'train_bene_df' is not defined

In [ ]:
train_bene_df['Patient_Age_Year'] = train_bene_df['DOB'].dt.year
train_bene_df['Patient_Age_Month'] = train_bene_df['DOB'].dt.month

In [ ]:
bene_age_year_df = pd.DataFrame(train_bene_df['Patient_Age_Year'].value_counts()).reset_index(drop=False)
bene_age_year_df.columns= ['year','num_of_beneficiaries']
bene_age_year_df = bene_age_year_df.sort_values(by='year')

In [ ]:
# Here, I'm displaying the distribution of BENEFICIARIES on the basis of their YEAR of Birth 
with plt.style.context('seaborn'):
  plt.figure(figsize=(21,9))
  fig = sns.barplot(data=bene_age_year_df, x='year', y='num_of_beneficiaries', palette='inferno')
  # Using the "patches" function we will get the location of the rectangle bars from the graph.
  ## Then by using those location(width & height) values we will add the annotations
  for p in fig.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    fig.annotate(f'{str(round((height*100)/train_bene_df.shape[0],1))+"%"}', (x + width/2, y + height*1.025), ha='center', fontsize=13.5, rotation=90)
  # Providing the labels and title to the graph
  plt.xlabel("\nBeneficary YEAR of Birth", fontdict=label_font_dict)
  plt.xticks(rotation=90)
  plt.ylabel("Number or % share of patients\n", fontdict=label_font_dict)
  plt.minorticks_on()
  plt.grid(which='major', linestyle="--", color='lightgrey')
  plt.title("Distribution of BENEFICIARIES based on their YEAR of birth\n", fontdict=title_font_dict)

In [ ]:
outputCollection2 = Helpers.createOutputCollection(context)
imgChart2 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle='Distribution of BENEFICIARIES based on their YEAR of birth',
                                                    plt=plt)

outputCollection2.addTemplateOutput(imgChart2)

In [ ]:
Helpers.save(context)

In [ ]:
bene_age_month_df = pd.DataFrame(train_bene_df['Patient_Age_Month'].value_counts()).reset_index(drop=False)
bene_age_month_df.columns= ['month','num_of_beneficiaries']
bene_age_month_df = bene_age_month_df.sort_values(by='month')

In [ ]:
# Here, I'm displaying the distribution of BENEFICIARIES on the basis of their MONTH of Birth 
with plt.style.context('seaborn'):
  plt.figure(figsize=(12,8))
  fig = sns.barplot(data=bene_age_month_df, x='month', y='num_of_beneficiaries', palette='summer')
  # Using the "patches" function we will get the location of the rectangle bars from the graph.
  ## Then by using those location(width & height) values we will add the annotations
  for p in fig.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    fig.annotate(f'{str(round((height*100)/train_bene_df.shape[0],1))+"%"}', (x + width/2, y + height*1.025), ha='center', fontsize=13.5, rotation=90)
  # Providing the labels and title to the graph
  plt.xlabel("\nBeneficary MONTH of Birth\n", fontdict=label_font_dict)
  plt.xticks(rotation=90)
  plt.ylabel("Number or % share of patients", fontdict=label_font_dict)
  plt.minorticks_on()
  plt.grid(which='major', linestyle="--", color='lightgrey')
  plt.title("Distribution of BENEFICIARIES based on their MONTH of birth\n", fontdict=title_font_dict)

In [ ]:
outputCollection3 = Helpers.createOutputCollection(context)
imgChart3 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle='Distribution of BENEFICIARIES based on their MONTH of birth',
                                                    plt=plt)

outputCollection3.addTemplateOutput(imgChart3)

In [ ]:
Helpers.save(context)

In [ ]:
# 0 means ALIVE and 1 means DEAD
train_bene_df['Dead_or_Alive'] = train_bene_df['DOD'].apply(lambda val: 0 if val != val else 1)

In [ ]:
# Here, I'm displaying the distribution of whether BENEFICIARY is ALIVE or NOT?
with plt.style.context('seaborn'):
  plt.figure(figsize=(10,8))
  fig = train_bene_df['Dead_or_Alive'].value_counts().plot(kind='bar', color=['lightgreen','coral'])
  # Using the "patches" function we will get the location of the rectangle bars from the graph.
  ## Then by using those location(width & height) values we will add the annotations
  for p in fig.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    fig.annotate(f'{str(round((height*100)/train_bene_df.shape[0],2))+"%"}', (x + width/2, y + height*1.015), ha='center', fontsize=13.5)
  # Providing the labels and title to the graph
  plt.xlabel("Alive or Dead Status?", fontdict=label_font_dict)
  plt.xticks(labels=["ALIVE","DEAD"], ticks=[0,1], rotation=20)
  plt.ylabel("Number or % share of patients\n", fontdict=label_font_dict)
  plt.grid(which='major', linestyle="--", color='lightgrey')
  plt.minorticks_on()
  plt.title("Distribution of BENEFICIARIES based on Alive or Dead Status\n", fontdict=title_font_dict)

In [ ]:
outputCollection4 = Helpers.createOutputCollection(context)
imgChart4 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle='Distribution of BENEFICIARIES based on Alive or Dead Status',
                                                    plt=plt)

outputCollection4.addTemplateOutput(imgChart4)

In [ ]:
Helpers.save(context)

In [ ]:
train_bene_df['DOD'] = pd.to_datetime(train_bene_df['DOD'])

In [ ]:
# Greatest Date of Death in the TRAIN set for beneficiaries
max_bene_DOD = max(train_bene_df['DOD'].unique()[1:])
# For all NAN DODs filling the greatest Date of Death
train_bene_df['DOD'].fillna(value=max_bene_DOD, inplace=True)
train_bene_df['AGE'] = np.round(((train_bene_df['DOD'] - train_bene_df['DOB']).dt.days)/365.0,1)
train_bene_df.drop(labels=['DOD'],axis=1,inplace=True)

In [ ]:
# Here, I'm displaying the distribution of AGE of Beneficiaries?
with plt.style.context('seaborn'):
  plt.figure(figsize=(10,8))
  train_bene_df['AGE'].plot(kind='hist', color='purple')
  # Providing the labels and title to the graph
  plt.xlabel("\nBeneficiaries Age in years", fontdict=label_font_dict)
  plt.ylabel("Frequency of patients\n", fontdict=label_font_dict)
  plt.grid(which='major', linestyle="--", color='lightgrey')
  plt.minorticks_on()
  plt.title("Distribution of BENEFICIARIES AGE", fontdict=title_font_dict)
  plt.legend();

In [ ]:
outputCollection5 = Helpers.createOutputCollection(context)
imgChart5 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle='Distribution of BENEFICIARIES AGE',
                                                    plt=plt)

outputCollection5.addTemplateOutput(imgChart5)

In [ ]:
Helpers.save(context)

In [ ]:
def bene_age_brackets(val):
  if val >=1 and val <=40:
    return 'Young'
  elif val > 40 and val <=60:
    return 'Mid'
  elif val > 60 and val <= 80:
    return 'Old'
  else:
    return 'Very Old'

In [ ]:
train_bene_df['AGE_groups'] = train_bene_df['AGE'].apply(lambda age: bene_age_brackets(age))

In [ ]:
age_grps = list(train_bene_df['AGE_groups'].unique())
for grp in age_grps:
  # Here, I'm displaying the distribution of AGE GROUPS of Beneficiaries?
  with plt.style.context('seaborn'):
    plt.figure(figsize=(8,6))
    train_bene_df[train_bene_df['AGE_groups'] == grp]['AGE'].plot(kind='hist', color='grey')
    # Providing the labels and title to the graph
    plt.xlabel("\nBeneficiaries Age in years", fontdict=label_font_dict)
    plt.ylabel("Frequency of patients\n", fontdict=label_font_dict)
    plt.grid(which='major', linestyle="--", color='lightgrey')
    plt.minorticks_on()
    plt.title("Distribution of BENEFICIARIES Age group -- {}".format(str(grp).upper()), fontdict=title_font_dict)
    plt.legend();

In [ ]:
outputCollection6 = Helpers.createOutputCollection(context)
imgChart6 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle='Distribution of BENEFICIARIES Age group',
                                                    plt=plt)

outputCollection6.addTemplateOutput(imgChart6)

In [ ]:
Helpers.save(context)

In [ ]:
# Here, I'm displaying the distribution of BENEFICIARIES on the basis of Human RACE
with plt.style.context('seaborn'):
  plt.figure(figsize=(10,8))
  fig = train_bene_df['Race'].value_counts().plot(kind='bar', color=['lightgreen','coral','purple','red'])
  # Using the "patches" function we will get the location of the rectangle bars from the graph.
  ## Then by using those location(width & height) values we will add the annotations
  for p in fig.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    fig.annotate(f'{str(round((height*100)/train_bene_df.shape[0],2))+"%"}', (x + width/2, y + height*1.02), ha='center', fontsize=13.5)
  # Providing the labels and title to the graph
  plt.xlabel("Human RACE", fontdict=label_font_dict)
  plt.xticks(labels=["Race_1","Race_2","Race_3","Race_5"], ticks=[0,1,2,3], rotation=10)
  plt.ylabel("Number or % share of patients\n", fontdict=label_font_dict)
  plt.grid(which='major', linestyle="--", color='lightgrey')
  plt.minorticks_on()
  plt.title("Distribution of BENEFICIARIES on the basis of Human RACE\n", fontdict=title_font_dict)

In [ ]:
outputCollection7 = Helpers.createOutputCollection(context)
imgChart7 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle='Distribution of BENEFICIARIES Age group',
                                                    plt=plt)

outputCollection7.addTemplateOutput(imgChart7)

In [ ]:
Helpers.save(context)

train_bene_df['Gender'] = train_bene_df['Gender'].apply(lambda val: 0 if val == 2 else 1)

# Here, I'm displaying the distribution of BENEFICIARIES on the basis of GENDER 
with plt.style.context('seaborn'):
    plt.figure(figsize=(10,8))
    fig = train_bene_df['Gender'].value_counts().plot(kind='bar', color=['yellow','purple'])
    # Using the "patches" function we will get the location of the rectangle bars from the graph.
    ## Then by using those location(width & height) values we will add the annotations
    for p in fig.patches:
        width = p.get_width()
        height = p.get_height()
        x, y = p.get_xy()
        fig.annotate(f'{str(round((height*100)/train_bene_df.shape[0],2))+"%"}', (x + width/2, y + height*1.015), ha='center', fontsize=13.5)
    # Providing the labels and title to the graph
    plt.xlabel("Gender Code", fontdict=label_font_dict)
    plt.ylabel("Number or % share of patients\n", fontdict=label_font_dict)
    plt.grid(which='major', linestyle="--", color='lightgrey')
    plt.minorticks_on()
    plt.title("Distribution of BENEFICIARIES based on GENDER\n", fontdict=title_font_dict)

outputCollection = Helpers.createOutputCollection(context)
imgChart = Helpers.createTemplateOutputPlotlibChart(context, chartTitle='Gender', plt=plt)

outputCollection.addTemplateOutput(imgChart)

Helpers.cre

Helpers.save(context)